In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# Grabbing a bunch of tech stocks for our portfolio
# symbol = 'WIKI/TSLA'
# tsla_df = web.DataReader(symbol, 'quandl', '2017-06-01', '2021-06-01',0,api_key='VyMgAh2xwHRz5fEhZH4-')
# symbol = 'WIKI/QCOM'
# qcom_df = web.DataReader(symbol, 'quandl', '2017-06-01', '2021-06-01',0,api_key='VyMgAh2xwHRz5fEhZH4-')
# symbol = 'WIKI/NKE'
# nke_df = web.DataReader(symbol, 'quandl', '2017-06-01', '2021-06-01',0,api_key='VyMgAh2xwHRz5fEhZH4-')
# symbol = 'WIKI/DIS'
# dis_df = web.DataReader(symbol, 'quandl', '2017-06-01', '2021-06-01',0,api_key='VyMgAh2xwHRz5fEhZH4-')
# symbol = 'WIKI/JNJ'
# jnj_df = web.DataReader(symbol, 'quandl', '2017-06-01', '2021-06-01',0,api_key='VyMgAh2xwHRz5fEhZH4-')

In [ ]:
# Grabbing a bunch of tech stocks for our portfolio
# aapl = quandl.get('WIKI/TSLA',start_date=start,end_date=end)
# import quandl


# start = pd.to_datetime('2017-06-01')
# end = pd.to_datetime('2021-06-01')

In [ ]:
# Download and get Daily Returns
tsla = pd.read_csv('./data/WIKI-TSLA.csv',index_col='Date',parse_dates=True)
qcom = pd.read_csv('./data/WIKI-QCOM.csv',index_col='Date',parse_dates=True)
nke = pd.read_csv('./data/WIKI-NKE.csv',index_col='Date',parse_dates=True)
dis = pd.read_csv('./data/WIKI-DIS.csv',index_col='Date',parse_dates=True)
jnj = pd.read_csv('./data/WIKI-JNJ.csv',index_col='Date',parse_dates=True)

In [ ]:
stocks = pd.concat([tsla['Adj. Close'],qcom['Adj. Close'],nke['Adj. Close'],dis['Adj. Close'],jnj['Adj. Close']],axis=1)
stocks.columns = ['tsla','qcom','nke','dis', 'jnj']

Created dataframe with the closeing prices

In [ ]:
stocks.head()

In [ ]:
stocks.pct_change(1).mean()

In [ ]:
stocks.pct_change(1).corr()

In [ ]:
stocks.pct_change(1).head()

In [ ]:
log_ret = np.log(stocks/stocks.shift(1))
log_ret.head()

In [ ]:
log_ret.hist(bins=100,figsize=(12,8))
plt.tight_layout()

In [ ]:
log_ret.mean()

In [ ]:
log_ret.cov() * 252

In [ ]:
np.random.seed(101)

num_ports = 5000

all_weights = np.zeros((num_ports, len(stocks.columns)))
ret_arr = np.zeros(num_ports)
vol_arr = np.zeros(num_ports)
sharpe_arr = np.zeros(num_ports)

for ind in range(num_ports):

    print(stocks.columns)

    # Weights
    weights = np.array(np.random.random(4))
    weights = weights/np.sum(weights)
    
    # Save Weights
    all_weights[ind:] = weights
    
    print("Expected Portfolio Return")
    ret_arr[ind] = np.sum( (log_ret.mean() * weights) * 252)

    print("Expected Volatility")
    vol_arr[ind] = np.sqrt(np.dot(weights.T,np.dot(log_ret.cov()*252, weights)))

    print("Sharpe Ratio")
    sharpe_arr[ind] = ret_arr[ind]/vol_arr[ind]

In [ ]:
sharpe_arr.max()

In [ ]:
sharpe_arr.argmax()

In [ ]:
all_weights[1420,:]

In [ ]:
max_sr_ret = ret_arr[1420]
max_sr_vol = vol_arr[1420]

In [ ]:
plt.figure(figsize=(12,8))
plt.scatter(vol_arr,ret_arr,c=sharpe_arr,cmap='plasma')
plt.colorbar(label='Sharpe Ratio')
plt.xlabel = 'Volatility'
plt.ylabel = 'Return'

plt.scatter(max_sr_vol,max_sr_ret,c='red',s=50,edgecolors='black')

In [ ]:
def get_ret_vol_sr(weights):
    weights = np.array(weights)
    ret = np.sum(log_ret.mean() * weights) * 252
    vol = np.sqrt(np.dot(weights.T, np.dot(log_ret.cov()*252,weights)))
    sr=ret/vol
    return np.array([ret,vol,sr])

In [ ]:
from scipy.optimize import minimize

In [ ]:
def neg_sharpe(weights):
    return get_ret_vol_sr(weights)[2] * -1

In [ ]:
def check_sum(weights):
    #return 0 if the sum of the weights is 1
    return np.sum(weights) - 1

In [ ]:
cons = ({'type':'eq','fun':check_sum})

In [ ]:
bounds = ((0,1),(0,1),(0,1),(0,1))

In [ ]:
init_guess = [0.25,0.25,0.25,0.25]

In [ ]:
opt_results = minimize(neg_sharpe,init_guess,method='SLSQP',bounds=bounds,constraints=cons)

In [ ]:
opt_results

In [ ]:
opt_results.x

In [ ]:
get_ret_vol_sr(opt_results.x)

In [ ]:
frontier_y = np.linspace(0,0.3,100)

In [ ]:
def minimize_volatility(weights):
    return get_ret_vol_sr(weights)[1]

In [ ]:
frontier_volatility = []

for possible_return in frontier_y:
    cons = ({'type':'eq', 'fun':check_sum},
           {'type':'eq','fun':lambda w: get_ret_vol_sr(w)[0]-possible_return})
    
    results = minimize(minimize_volatility,init_guess,method='SLSQP',bounds=bounds,constraints=cons)
    
    frontier_volatility.append(results['fun'])

In [ ]:
plt.figure(figsize=(12,8))
plt.scatter(vol_arr,ret_arr,c=sharpe_arr,cmap='plasma')
plt.colorbar(label='Sharpe Ratio')
plt.xlabel = 'Volatility'
plt.ylabel = 'Return'

plt.plot(frontier_volatility, frontier_y, 'g--',linewidth=3)